In [ ]:
%pip install torchvision

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor    

In [ ]:
class SupervisedKernel(nn.Module):
    def __init__(self, kernel: nn.Module = None, supervisor: nn.Module = None):
        super(SupervisedKernel, self).__init__()
        if(kernel is not None):
            self.kernel = kernel
        else:
            self.kernel = nn.Sequential(nn.Linear(8, 8), nn.ReLU())
        self.superviser = supervisor
        if(self.superviser is not None):
            self.superviser.requires_grad_(False)

    def forward(self, x):
        y = self.kernel(x)
        if(self.superviser is not None):
            target = self.superviser(x)
        else:
            target = y
        return y, target
    
    @classmethod
    def is_supervised(self, module: nn.Module):
        return isinstance(module, SupervisedKernel) and (module.superviser is not None)
    
    def update(self, kernel: nn.Module, retrain: bool = True):
        # прежнее ядро становится супервизором
        self.superviser = self.kernel.requires_grad_(False)
        self.kernel = kernel

    def advise(self, x):
        

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [8]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [9]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.152025  [   64/60000]
loss: 1.157322  [ 6464/60000]
loss: 0.973880  [12864/60000]
loss: 1.106955  [19264/60000]
loss: 0.987002  [25664/60000]
loss: 1.013624  [32064/60000]
loss: 1.050538  [38464/60000]
loss: 1.008866  [44864/60000]
loss: 1.027371  [51264/60000]
loss: 0.971470  [57664/60000]
Test Error: 
 Accuracy: 66.2%, Avg loss: 0.979064 

Epoch 2
-------------------------------
loss: 1.033243  [   64/60000]
loss: 1.059224  [ 6464/60000]
loss: 0.860504  [12864/60000]
loss: 1.016855  [19264/60000]
loss: 0.900433  [25664/60000]
loss: 0.919633  [32064/60000]
loss: 0.974077  [38464/60000]
loss: 0.935530  [44864/60000]
loss: 0.949180  [51264/60000]
loss: 0.904585  [57664/60000]
Test Error: 
 Accuracy: 67.5%, Avg loss: 0.907046 

Epoch 3
-------------------------------
loss: 0.946664  [   64/60000]
loss: 0.990998  [ 6464/60000]
loss: 0.779232  [12864/60000]
loss: 0.953026  [19264/60000]
loss: 0.842993  [25664/60000]
loss: 0.851109  [32064/600

In [17]:
X = torch.rand(4, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8, 8, 6, 8], device='cuda:0')
